In [0]:
!mkdir .kaggle

In [0]:
!ls

In [0]:
import json
token = {"username":"yjtrxj","key":"131bbafb775c8139a4bb07dbc9427ac7"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [0]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [0]:
!kaggle config set -n path -v{/content}

In [0]:
!chmod 600 /root/.kaggle/kaggle.json

In [0]:
!kaggle datasets list -s blog

In [0]:
!kaggle datasets download -d rtatman/blog-authorship-corpus -p /content/

In [0]:
!unzip \blog-authorship-corpus.zip

In [0]:
import pandas as pd
blog = pd.read_csv('blogtext.csv')

In [0]:
#blog_df = blog.sample(frac =0.012,replace=False,random_state=5678)

In [0]:
blog_df = blog.iloc[:8000,:]

In [0]:
blog_df.shape

In [0]:
blog_df.isnull().sum()

In [0]:
blog_df['text'].unique()

In [0]:
blog_df.info()

In [0]:
blog_df['label'] = blog_df.apply(lambda r: [r['gender'], str(r['age']), r['topic'],r['sign']], axis=1)

In [0]:
blog_df.drop(columns=['id','gender','age','topic','sign','date'],inplace = True)

In [0]:
blog_df.head(2)

In [0]:
from nltk.corpus import stopwords
import re
import nltk
nltk.download('stopwords')
blog_df['text'] = blog_df['text'].map(lambda s : s.lower())
blog_df['text'] = blog_df['text'].map(lambda s:re.sub('[0-9+-/#()!:\']','',s))
blog_df['text'] = blog_df['text'].map(lambda s :s.strip())
sw = stopwords.words('english')
blog_df['text'] = blog_df['text'].apply(lambda s: ' '.join([word for word in s.split() if word not in (sw)]))
blog_df['text']

In [0]:
x = blog_df.text
y = blog_df.label

In [0]:
#you can change the test size & a smaller train size will help your system to fit in count vectorizer otherwise system will crash
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [0]:
x_train,x_val,y_train,y_val = train_test_split(x_train,y_train,test_size=0.3)

In [0]:
print("X_train shape is : ",x_train.shape)
print("X_test shape is : ",x_test.shape)
print("X_val shape is : ",x_val.shape)
print("y_train shape is : ",y_train.shape)
print("y_test shape is : ",y_test.shape)
print("y_val shape is : ",y_val.shape)

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

In [0]:
cv = CountVectorizer(ngram_range=(1,2))

In [0]:
x_train = cv.fit_transform(x_train)

In [0]:
x_test = cv.transform(x_test)

In [0]:
x_val = cv.transform(x_val)

In [0]:
#just to check the
print(x_val)

In [0]:
label_counts = dict()
for labels in blog_df.label.values:
    for label in labels:
        if label in label_counts:
            label_counts[label] += 1
        else:
            label_counts[label] = 1

In [0]:
label_counts

In [0]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y_train = mlb.fit_transform(y_train)
y_test = mlb.transform(y_test)
y_val = mlb.transform(y_val)

In [0]:
y_test.shape

In [0]:
y_train.shape

In [0]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

clf = LogisticRegression(solver='lbfgs',max_iter=10000)
clf = OneVsRestClassifier(clf)
clfm = OneVsRestClassifier(MultinomialNB())
clf0 = GridSearchCV(DecisionTreeClassifier(),cv=5,param_grid={'max_depth':[80,250,500,750,1000]})
clf1 = GridSearchCV(KNeighborsClassifier(),cv=5,param_grid={'n_neighbors':[5,10,15,20,25]})
clf2 = GridSearchCV(RandomForestClassifier(),cv =5,param_grid={'max_depth':[80,250,500,750,1000]})

In [0]:
%%time
clf.fit(x_train,y_train)

In [0]:
y_pred = clf.predict(x_test)

In [0]:
from sklearn.metrics import hamming_loss, accuracy_score,classification_report
print("Model accuracy is:", clf.score(x_train,y_train))
print("Accuracy is:",accuracy_score(y_test,y_pred))
print("Hamming_loss:", hamming_loss(y_test, y_pred))
report = classification_report(y_test,y_pred)
print(report)

In [0]:
y_test_re = mlb.inverse_transform(y_test)

In [0]:
y_pred_re = mlb.inverse_transform(y_pred)

In [0]:
label_table = pd.DataFrame({'Predicted':y_pred_re,'Actual':y_test_re})

In [0]:
label_table.head(6)

In [0]:
clfm.fit(x_train,y_train)

In [0]:
y_predm = clfm.predict(x_test)

In [0]:
from sklearn.metrics import hamming_loss, accuracy_score,classification_report
print("Model accuracy is:", clfm.score(x_train,y_train))
print("Accuracy is:",accuracy_score(y_test,y_predm))
print("Hamming_loss:", hamming_loss(y_test, y_predm))
reportm = classification_report(y_test,y_predm)
print(reportm)

In [0]:
clf0.fit(x_train,y_train)
print("This is the best score :",clf0.best_score_)
print("This are the best parameters:",clf0.best_params_)
print("This is the best estimator:",clf0.best_estimator_)

In [0]:
lr0 = clf0.best_estimator_
lr0.fit(x_train,y_train)

In [0]:
y_pred0 = lr0.predict(x_test)

In [0]:
from sklearn.metrics import hamming_loss, accuracy_score,classification_report
print("Model accuracy is:", lr0.score(x_train,y_train))
print("Accuracy is:",accuracy_score(y_test,y_pred0))
print("Hamming_loss:", hamming_loss(y_test, y_pred0))
report0 = classification_report(y_test,y_pred0)
print(report0)

In [0]:
y_pred_re0 = mlb.inverse_transform(y_pred0)

In [0]:
label_table0 = pd.DataFrame({'Predicted':y_pred_re0,'Actual':y_test_re})

In [0]:
label_table0.head(6)

In [0]:
x_train.shape

In [0]:
y_train.shape

In [0]:
clf1.fit(x_train,y_train)
print("This is the best score :",clf1.best_score_)
print("This are the best parameters:",clf1.best_params_)
print("This is the best estimator:",clf1.best_estimator_)

In [0]:
lr1 = clf1.best_estimator_
lr1.fit(x_train,y_train)

In [0]:
y_pred1 = lr1.predict(x_test)

In [0]:
from sklearn.metrics import hamming_loss, accuracy_score,classification_report
print("Model accuracy is:", lr1.score(x_train,y_train))
print("Accuracy is:",accuracy_score(y_test,y_pred1))
print("Hamming_loss:", hamming_loss(y_test, y_pred1))
report1 = classification_report(y_test,y_pred1)
print(report1)

In [0]:
y_pred_re1 = mlb.inverse_transform(y_pred1)

In [0]:
label_table1 = pd.DataFrame({'Predicted':y_pred_re1,'Actual':y_test_re})

In [0]:
clf2.fit(x_train,y_train)
print("This is the best score :",clf2.best_score_)
print("This are the best parameters:",clf2.best_params_)
print("This is the best estimator:",clf2.best_estimator_)

In [0]:
lr2 = clf2.best_estimator_
lr2.fit(x_train,y_train)

In [0]:
y_pred2 = lr2.predict(x_test)

In [0]:
from sklearn.metrics import hamming_loss, accuracy_score,classification_report
print("Model accuracy is:", lr2.score(x_train,y_train))
print("Accuracy is:",accuracy_score(y_test,y_pred2))
print("Hamming_loss:", hamming_loss(y_test, y_pred2))
report2 = classification_report(y_test,y_pred2)
print(report2)

In [0]:
y_pred_re2 = mlb.inverse_transform(y_pred2)

In [0]:
label_table2 = pd.DataFrame({'Predicted':y_pred_re2,'Actual':y_test_re})

In [0]:
label_table2.head()